<a href="https://colab.research.google.com/github/C4viar/PPD_MSLD_2019_2020_LL-AL-MT/blob/master/Draft_code/projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 1

1. Reproduire avec l’implémentation
https://huggingface.co/transformers/model_doc/camembert.html
les manipulations décrites dans https://github.com/pytorch/fairseq/tree/master/examples/camembert

## CamemBERT

Rappel : Tous les types de modèles : https://huggingface.co/models

In [0]:
! pip install transformers;

#### Fill mask

In [30]:
from transformers import pipeline, CamembertModel, CamembertTokenizer, CamembertForMaskedLM, PreTrainedModel


sentence_fillmask = """Le camembert c'est <mask>""".lstrip()
model_fillmask_str = "camembert-base"
model_fillmask = CamembertForMaskedLM.from_pretrained(model_fillmask_str )
tokenizer_fillmask = CamembertTokenizer.from_pretrained(model_fillmask_str )


fill_mask = pipeline(
    "fill-mask",
    model = model_fillmask,
    tokenizer = tokenizer_fillmask 
    )

fill_mask(sentence_fillmask)

[{'score': 0.2072705626487732,
  'sequence': "<s> Le camembert c'est quoi</s>",
  'token': 484},
 {'score': 0.08772061765193939,
  'sequence': "<s> Le camembert c'est bon</s>",
  'token': 212},
 {'score': 0.05717896297574043,
  'sequence': "<s> Le camembert c'est délicieux</s>",
  'token': 7200},
 {'score': 0.05560891702771187,
  'sequence': "<s> Le camembert c'est...</s>",
  'token': 186},
 {'score': 0.05323265120387077,
  'sequence': "<s> Le camembert c'est :</s>",
  'token': 43}]

#### Sentiment analysis

Indisponible avec Camembert

In [0]:
from transformers import CamembertForTokenClassification

sentence_sentiment = "Le camembert c'est bon"

model_sentiment_str = ""
model_sentiment = CamembertForSequenceClassification.from_pretrained(model_sentiment_str)
tokenizer_sentiment = CamembertTokenizer.from_pretrained(model_sentiment_str )

sentiment_analysis = pipeline(
    "sentiment-analysis",
    model = model_sentiment,
    tokenizer = tokenizer_sentiment
    )

sentiment_analysis(sentence_sentiment)

#### NER

Indisponible avec Camembert

In [0]:
from transformers import CamembertForTokenClassification

sentence_ner = "Amar habite à Paris"

ner_model_str = ''
ner_model = CamembertForTokenClassification.from_pretrained(ner_model_str)
tokenizer_ner= CamembertTokenizer.from_pretrained(ner_model_str)

ner = pipeline(
    "ner",
    model = ner_model,
    tokenizer = tokenizer_ner
    )

ner(sentence_net)

#### Question answering

In [0]:
from transformers import CamembertForTokenClassification

quest_model_str = ''
quest_model = XXXXXXX.from_pretrained(quest_model_str)  # XXXXXXX à remplacer / ! \
tokenizer_quest= CamembertTokenizer.from_pretrained(quest_model_str)

quest = pipeline(
    "question_answering",
    model = quest_model,
    tokenizer = tokenizer_quest
    )

quest(
    'question' : "Où habite Amar ? ",
    'context' : "Amar habite à Paris.
)

#### Translation

#### Feature extraction

In [0]:
feature_extraction = pipeline(
    "feature-extraction",
    model = model_camembert,
    tokenizer = tokenizer_camembert
    )

feature_extraction(sentence)

#### Extract All-layers

In [0]:
input_ids = torch.tensor(tokenizer.encode(sentence, add_special_tokens=True)).unsqueeze(0)
labels = torch.tensor([1] * input_ids.size(1)).unsqueeze(0)

In [0]:
outputs_camembert = model(input_ids)

In [0]:
outputs_cammebert

(tensor([[[ 20.6235,  -4.2011,   6.8420,  ...,  -5.7573,  -3.1165,   1.1391],
          [  2.6217,  -5.2430,  17.5290,  ..., -14.2041,  -2.4229,   0.8586],
          [ -0.1691,  -9.8924,   1.8213,  ..., -22.9433, -11.8068,  -1.9409],
          ...,
          [ -0.6569, -10.2129,  -0.5032,  ...,  -7.7102, -10.5380,   1.3649],
          [ -1.6542,  -4.3773,   2.8810,  ...,  -7.5383,  -3.0471,  -3.1600],
          [  6.3686,  -5.1610,  23.3708,  ...,  -7.9306,  -5.1287,   1.9971]]],
        grad_fn=<AddBackward0>),)

## FlauBERT
### Ne fonctionne pas pour le moment

In [0]:
from transformers import FlaubertModel, AutoTokenizer, AutoModelWithLMHead

flaubert = "flaubert-large-cased"

tokenizer_flaubert = AutoTokenizer.from_pretrained(flaubert)
model_flaubert = AutoModelWithLMHead.from_pretrained(flaubert)


fill_mask = pipeline(
    "fill-mask",
    model = flaubert,
    tokenizer = flaubert
)

fill_mask(sentence)

### Flaubert : essai 2

In [0]:

import torch
from transformers import FlaubertModel, FlaubertTokenizer

# Choose among ['flaubert-small-cased', 'flaubert-base-uncased', 'flaubert-base-cased', 'flaubert-large-cased']
modelname = 'flaubert-base-cased' 

# Load pretrained model and tokenizer
flaubert, log = FlaubertModel.from_pretrained(modelname, output_loading_info=True)
flaubert_tokenizer = FlaubertTokenizer.from_pretrained(modelname, do_lowercase=False)
# do_lowercase=False if using cased models, True if using uncased ones

sentence = "Le camembert c'est <mask>."
token_ids = torch.tensor([flaubert_tokenizer.encode(sentence)])

fill_mask = pipeline(
    "fill-mask",
    model = flaubert,
    tokenizer = flaubert_tokenizer
)

fill_mask(sentence)

#last_layer = flaubert(token_ids)[0]
#print(last_layer.shape)
# torch.Size([1, 8, 768])  -> (batch size x number of tokens x embedding dimension)

# The BERT [CLS] token correspond to the first hidden state of the last layer
#cls_embedding = last_layer[:, 0, :]